In [ ]:
!unzip /content/drive/MyDrive/Dataset_Final.zip

In [ ]:
folder1=r"/content/Dataset/Bus 2"
folder2=r"/content/Dataset/Non Buses(Negative Examples)/data/vehicles"
folder3=r"/content/Dataset/Non Buses(Negative Examples)/data/non-vehicles"
DIRECTORY=r'D:\SDP'
CATEGORIES=['bus','non-bus']

In [ ]:
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import imutils
import easyocr

In [ ]:
IMG_SIZE=100

data=[]
for category in CATEGORIES:
    folder=os.path.join(DIRECTORY,category)
    label=CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path=os.path.join(folder,img)
        img_arr=cv2.imread(img_path)
        if img_arr is None:
            print('Wrong path:', img_path)
        else:
            img_arr=cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
            data.append([img_arr,label])

In [ ]:
len(data)

In [ ]:
random.shuffle(data)

In [ ]:
X=[]
y=[]

for features,labels in data:
    X.append(features)
    y.append(labels) 

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
pickle.dump(X,open('X.pkl','wb'))
pickle.dump(y,open('y.pkl','wb'))

In [ ]:
X1=pickle.load(open('X.pkl','rb'))
y1=pickle.load(open('y.pkl','rb'))

In [ ]:
X1=X1/255

In [ ]:
X1.shape

In [ ]:
model=Sequential()

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(128, input_shape=X1.shape[1:], activation='relu'))

model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
NAME='Classify Buses-{int(time.time())}'
tensorboard=TensorBoard(log_dir=f'logs\\{NAME}\\')

In [ ]:
model.fit(X1,y1, epochs=5, validation_split=0.1,callbacks=[tensorboard])

In [ ]:
img = cv2.imread(r"D:\\Final Dataset\Numbered Bus\35.jpeg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [ ]:
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
edged = cv2.Canny(bfilter, 30, 200) #Edge detection
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

In [ ]:
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

In [ ]:
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break

In [ ]:
location

In [ ]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0,255, -1)
new_image = cv2.bitwise_and(img, img, mask=mask)

In [ ]:
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

In [ ]:
(x,y) = np.where(mask==255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
cropped_image = gray[x1-3:x2+5, y1-3:y2+5]

In [ ]:
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
result

In [ ]:
text = result[0][-2]
font = cv2.FONT_HERSHEY_SIMPLEX
res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))

In [ ]:
text

In [ ]:
if text=='5':
    print("Wanowrie to Bibewadi")
if text=='4':
    print("Koregaon Park to MG Road")
if text=='3':
    print("Kondhwa to Fatima Nagar")
if text=='2':
    print("Salunke Vihar to Hadapsar")
if text=='1':
    print("Kalyani Nagar to Lohegaon")